In [1]:
import numpy as np
import tensorflow as tf
import gym
import scipy.signal
import os
import time
import inspect
from multiprocessing import Process
import tensorflow.contrib.layers as layers

/home/eshahrivar/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def atari_model(img_in, num_actions, scope, reuse=False):
    # as described in https://storage.googleapis.com/deepmind-data/assets/papers/DeepMindNature14236Paper.pdf
    with tf.variable_scope(scope, reuse=reuse):
        out = img_in
        with tf.variable_scope("convnet"):
            # original architecture
            out = layers.convolution2d(out, num_outputs=32, kernel_size=8, stride=4, activation_fn=tf.nn.relu)
            out = layers.convolution2d(out, num_outputs=64, kernel_size=4, stride=2, activation_fn=tf.nn.relu)
            out = layers.convolution2d(out, num_outputs=64, kernel_size=3, stride=1, activation_fn=tf.nn.relu)
        out = layers.flatten(out)
        with tf.variable_scope("action_value"):
            out = layers.fully_connected(out, num_outputs=512,         activation_fn=tf.nn.relu)
            out = layers.fully_connected(out, num_outputs=num_actions, activation_fn=None)

        return out

In [27]:
a=(1,2,3)
z=tf.placeholder(tf.uint8, [None] + list(a))
s=tf.placeholder(tf.float32,[1])

In [28]:
sess = tf.Session()

In [29]:
print(s.shape.dims)

[Dimension(1)]


In [30]:
input = tf.constant([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
a=tf.reduce_max(input,1)
for x in a.eval(session=sess):
    print(x)

3
6
9


In [33]:
a=tf.constant([1,2,3])
c=tf.constant([1,0,1])
dd=(1-c)*a
dd.eval(session=sess)

array([0, 2, 0], dtype=int32)

In [26]:
b=tf.constant([])
c=tf.constant([2])
d=tf.concat(b,c)

ValueError: Shapes (1,) and () are incompatible

In [33]:
obs_t_ph = tf.placeholder(tf.uint8, [None] + list((5,2,8)))
obs_t_float   = tf.cast(obs_t_ph,   tf.float32) / 255.0

In [34]:
x=atari_model(obs_t_float, 5, scope="q_current", reuse=False)

In [35]:
x.shape.dims

[Dimension(None), Dimension(5)]

In [ ]:
input_nn.shape.dims

In [ ]:
sess = tf.InteractiveSession()

In [5]:
a=tf.log([[100., 100.]])
print(a.eval())
b=tf.constant([[100, 100., 100]])
print(b.eval())
print (a.shape)
print(b.shape)
c=np.array([[2,2]])
print(c.shape)

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

In [6]:
samples = tf.multinomial(b, 5)
print (samples.eval())
print(samples.shape)

NameError: name 'b' is not defined

In [7]:
sy_ob_no = tf.placeholder(shape=[None, 3], name="ob", dtype=tf.float32)
print(sy_ob_no.shape[0])

?


In [8]:
z=tf.constant([2, 1],shape=[1,2])
print(z.eval())
s=tf.constant([2,2], shape=[2,1])
print(s.eval())
h=tf.matmul(s,z)
print(h.eval())

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

In [9]:
s=tf.random_normal(z)
print(s.eval())

ValueError: Shape must be rank 1 but is rank 2 for 'random_normal/RandomStandardNormal' (op: 'RandomStandardNormal') with input shapes: [1,2].

In [78]:
a=tf.reduce_mean(s)
print(a.eval())

2


In [79]:
rew=np.array([1,2,3])
a=[]
for s in rew:
    a.append(s)
    print(s)
    
print(a)
print(a.reverse())

1
2
3
[1, 2, 3]
None


In [80]:
a=1
print(pow(a,2))

1


In [81]:
x = tf.constant([[1., 1.], [2., 2.]])
z=tf.reduce_mean(sy_ob_no)  # 1.5

In [82]:
sess.run(z, feed_dict={sy_ob_no: [[1,1,1]]})

1.0

In [83]:
a=np.array([1,2,3])

In [84]:
s=np.mean(a)

In [85]:
b=a-s

In [86]:
print(b)

[-1.  0.  1.]


In [87]:
sigma=np.std(a)

In [88]:
b=(a-s)/sigma

In [89]:
print(b)

[-1.22474487  0.          1.22474487]


In [90]:
print(sigma)

0.816496580927726


In [91]:
a=np.ones(5)
a.shape

(5,)

In [97]:
sy_mean = build_mlp(sy_ob_no, 2, "PG_C", n_layers=3, size=10)
sy_logstd = tf.Variable(tf.zeros([1,2]), name="logstd", dtype='float', trainable=True) # logstd should just be a trainable variable, not a network output.
sy_std = tf.exp(sy_logstd)

z = tf.random_normal(tf.constant([2,1]))
sy_sampled_ac=tf.matmul(sy_std, z)+sy_mean
sy_sampled_ac=tf.reshape(sy_sampled_ac,[-1])
sy_ac_na = tf.placeholder(shape=[None, 2], name="ac", dtype=tf.float32)


In [100]:
sqrt_sigma= tf.diag(sy_logstd)
sess.run(sqrt_sigma, feed_dict={sy_logstd:np.ones((1,2))})

array([[[[1., 0.]],

        [[0., 1.]]]], dtype=float32)

In [106]:
x=sy_mean-sy_ac_na
x.shape.dims
sy_mean.shape.dims

[Dimension(None), Dimension(2)]

In [107]:
z =  x/sy_std # Hint: Use the log probability under a multivariate gaussian
sy_logprob_n = 0.5*tf.norm(z)